In [ ]:
# 📦 1. Importar librerías
import pandas as pd

In [ ]:
# 💬 Instalar AutoGluon si es necesario
%pip install autogluon.timeseries

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [ ]:
# 📄 2. Cargar datasets desde carpeta local
import pandas as pd

BASE = "/kaggle/input/labo3-sales-data"

# i) Carga sin parsear fechas
df_sellin = pd.read_csv(
    f"{BASE}/sell-in.txt",
    sep="\t",
)
df_productos = pd.read_csv(f"{BASE}/tb_productos.txt", sep="\t")


In [ ]:
# 📄 Leer lista de productos a predecir
with open(f"{BASE}/780_a_predecir.txt", "r") as f:
    product_ids = [
        int(line.strip())
        for line in f
        if line.strip().isdigit()
    ]


In [ ]:
# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [ ]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

In [ ]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [ ]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [ ]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [ ]:
# ⏰ 4. Crear TimeSeriesDataFrame

!pip install autogluon.timeseries

from autogluon.timeseries import TimeSeriesDataFrame

ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [ ]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [ ]:
!pip install autogluon.timeseries

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

# 1) Creamos el predictor sin presets en __init__
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS',
    path='modelo_autogluon_guardado' 
)

# 2) Entrenamos indicando presets en fit()
predictor.fit(
    ts_data,
    presets='best_quality',   # aquí van los presets
    num_val_windows=5,
    time_limit=3600,      
    random_seed=777
)



In [ ]:
# 💾 Guardar el modelo entrenado para no perderlo
predictor.save()


In [ ]:
# 📊 Leaderboard de modelos entrenados
leaderboard_df = predictor.leaderboard(ts_data, silent=True)

# Mostrar top 10 modelos
print("🏆 Top 10 modelos:")
display(leaderboard_df.head(10))

# 💾 Guardar leaderboard completo en CSV
leaderboard_df.to_csv("leaderboard_autogluon.csv", index=False)


In [ ]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

In [ ]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

In [ ]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


In [ ]:
# 💾 7. Guardar archivo
#resultado.to_csv("PREDICCION_CON_AUTOGLUON_2.csv", index=False)

#resultado.head()


In [ ]:
# 🔁 Obtener modelos individuales, excluyendo el WeightedEnsemble
modelos = predictor.leaderboard(silent=True)['model'].tolist()
modelos_disponibles = [m for m in modelos if "Ensemble" not in m]

# Selección de los tres mejores
modelos_top3 = modelos_disponibles[:3]
print("Modelos seleccionados para ensemble:", modelos_top3)

In [ ]:
# 🎯 Predicciones de los tres mejores modelos
pred1 = predictor.predict(ts_data, model=modelos_top3[0])
pred2 = predictor.predict(ts_data, model=modelos_top3[1])
pred3 = predictor.predict(ts_data, model=modelos_top3[2])


In [ ]:
# 🤝 Ensemble promedio
pred_final = (pred1 + pred2 + pred3) / 3


In [ ]:
# 👇 pred_final ya es un DataFrame con timestamp como índice
df_pred = pred_final.reset_index()  # ✅ convierte índices en columnas

# Filtramos solo febrero 2020
df_febrero = df_pred[df_pred['timestamp'] == '2020-02-01']

# Seleccionamos columnas y renombramos
df_feb = df_febrero[['item_id', 'mean']].copy()
df_feb.columns = ['product_id', 'tn']


In [ ]:
# 💾 Guardar CSV para Kaggle
df_feb.to_csv("submission_ensemble_feb2020.csv", index=False)
df_feb.head()
